In [ ]:
import numpy as np
import torch
import torch.utils.data as data
from PIL import Image
from skimage.transform import resize
import sys
import lmdb
import os
import pickle
import matplotlib.pyplot as plt
import imgaug as ia
from imgaug import augmenters as iaa
import cv2

sys.path.append(os.path.dirname(os.path.abspath('')))
from utils.directory import DATA_DIR, DATASET_DIR
from utils.directory import DATA_DIR, DATASET_DIR
import utils.prepare_data as pd
import utils.error as error
import utils.visualize as visual
import utils.convert_xyz_uvd as xyzuvd

In [ ]:
save_prefix = 'test_fpha'
REORDER = visual.REORDER
keys_cache_file = os.path.join(DATA_DIR, save_prefix + '_keys_cache.p')
keys = pickle.load(open(keys_cache_file, "rb"))
idx = 0
key = keys[idx]
dataroot_uvd_gt_scaled = os.path.join(DATA_DIR, save_prefix + '_uvd_gt_scaled.lmdb')
uvd_gt_scaled_env = lmdb.open(dataroot_uvd_gt_scaled, readonly=True, lock=False, readahead=False, meminit=False)
uvd_gt = pd.read_lmdb(key, uvd_gt_scaled_env, np.float32, (21, 3))

dataroot_xyz_gt = os.path.join(DATA_DIR, save_prefix + '_xyz_gt.lmdb')
xyz_gt_env = lmdb.open(dataroot_xyz_gt, readonly=True, lock=False, readahead=False, meminit=False)
xyz_gt = pd.read_lmdb(key, xyz_gt_env, np.float32, (21, 3))
uvd_gt_ori = xyzuvd.xyz2uvd_color(xyz_gt)

In [ ]:
img = np.asarray(Image.open(os.path.join(DATASET_DIR, 'First_Person_Action_Benchmark', 'Video_files', key)))
fig, ax = plt.subplots()
ax.imshow(img)
visual.visualize_joints_2d(ax, uvd_gt_ori[REORDER], joint_idxs=False)

In [ ]:
img = np.asarray(Image.open(os.path.join(DATASET_DIR, 'First_Person_Action_Benchmark', 'Video_files_256_crop', key)))
fig, ax = plt.subplots()
ax.imshow(img)
visual.visualize_joints_2d(ax, uvd_gt[REORDER], joint_idxs=False)

In [ ]:
skel_kps = []
for kps in uvd_gt:
    skel_kps.append(ia.Keypoint(x=kps[0],y=kps[1]))
skel_kpsoi = ia.KeypointsOnImage(skel_kps, shape=img.shape)

seq = iaa.Sequential([
    iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="HSV"),
    iaa.WithChannels(0, iaa.Add((-18, 18))), #hue
    iaa.ChangeColorspace(from_colorspace="HSV", to_colorspace="RGB"),
    iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}),
])

seq_det = seq.to_deterministic()
img_aug = seq_det.augment_images([img])[0]
kps_aug = seq_det.augment_keypoints([skel_kpsoi])[0]
kps_aug = kps_aug.get_coords_array()
uvd_gt_aug = np.concatenate((kps_aug, np.expand_dims((uvd_gt[:, 2]), -1)), -1)

In [ ]:
img = Image.open(os.path.join(DATASET_DIR, 'First_Person_Action_Benchmark', 'Video_files_256_crop', key))
fig, ax = plt.subplots()
ax.imshow(img_aug)
visual.visualize_joints_2d(ax, uvd_gt_aug[REORDER], joint_idxs=False)

In [ ]:
scoremap_gt = pd.create_multiple_gaussian_map(uvd_gt_scaled, (256, 256))
for i in range(21):
    fig, ax = plt.subplots()
    visual.visualize_joints_2d(ax, uvd_gt_aug[REORDER], joint_idxs=False)
    scoremap_show = cv2.cvtColor(scoremap_gt[:, :, i],cv2.COLOR_GRAY2RGB)
    scoremap_show = cv2.normalize(scoremap_show, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC3)
    show = 0.5*scoremap_show + 0.5*img_aug
    ax.imshow(show.astype('uint32'))
    ax.set_title('scoremap_gt')

In [ ]:
scoremap_gt = pd.create_multiple_gaussian_map(uvd_gt_scaled, (256, 256))
scoremap_gt = resize(scoremap_gt, (32, 32), order=3, preserve_range=True).astype('float32')
img_32_aug = pd.sk_resize(img_aug, (32, 32)).astype('uint32')
for i in range(21):
    fig, ax = plt.subplots()
    scoremap_show = cv2.cvtColor(scoremap_gt[:, :, i],cv2.COLOR_GRAY2RGB)
    scoremap_show = cv2.normalize(scoremap_show, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC3)
    show = 0.5*scoremap_show + 0.5*img_32_aug
    ax.imshow(show.astype('uint32'))
    ax.set_title('scoremap_gt')